## SENTIMENT ANALYSIS OF CITY OF LA JOB POSTINGS TO INCREASE THE DIVERSITY HIRING

**Challenge:**

The City of Los Angeles faces a big hiring challenge: one third of its 50,000 workers are eligible to retire by July of 2020. The city has partnered with Kaggle to create a competition to improve the job bulletins it will use to fill all those open positions.
The content, tone, and format of job descriptions can influence the quality of the applicant pool. Overly-specific requirements, for example, may discourage diversity. The Los Angeles Mayor’s Office wants to reimagine the city’s job bulletins by using text analysis to identify where they can improve.

**Goal:**
The goal is to convert a folder full of plain-text job postings into a structured CSV file and then to use this data to:
1. Identify language that can negatively bias the pool of applicants; 
2. Improve the diversity and quality of the applicant pool; 
3. Make it easier to determine which promotions are available to employees in each job class.

**BASICALLY, TURNING A PLAIN JOB POSTING TEXT INTO A MORE DIVERSITY ATTRACTING FLYER**
![Final job posting](https://storage.googleapis.com/kagglesdsdata/datasets/239825/508425/Job%20posting%20banner.png?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1561828368&Signature=X5%2BT8pPvV5DqEvdQDNTVRTG12ts8zjPY%2FkUS1ciuLGTwPfbHSeDW3JLWHTDVRa645vzBAaQ9t%2FWBV6FnjgABMwk2lT4Z68eO2sWhoItFN909HrL8GDc5gZh4%2B%2BPY7v6ILFJha579XoljQUSfo0OWmrnx%2FUIUgYiITu7dTKwlmGY0OmtCGrJMmrdbVZLUYJPcO6bT2aUnO2m0grk5Nm%2FIlPvHHCg%2FNkirlnzRDxKl10k5hqLKy%2FQ2RaweB8DFpgf0m%2B%2FsHUNVIDJHSJDFKQL%2FvqXLYNLBK0pYAXwObVZPSGLw2pG19OGp9ZV%2FzuAIBC3Fs5btby1XoIljVz7xMw7Pxw%3D%3D)


**The Process:**

[PART 1: JOB POSTING TEXT BULLETIN TO STRUCTURED CSV FILE:](https://www.kaggle.com/vicely07/sentiment-analysis-of-city-of-la-job-postings?scriptVersionId=16046935#PART-1:-JOB-POSTING-TEXT-BULLETIN-TO-STRUCTURED-CSV-FILE:):

[PART 2: RAW TEXT ANALYSIS ](https://www.kaggle.com/vicely07/sentiment-analysis-of-city-of-la-job-postings?scriptVersionId=16046935#PART-2:-RAW-TEXT-ANALYSIS)

[PART 3: SENTIMENT ANALYSIS](https://www.kaggle.com/vicely07/sentiment-analysis-of-city-of-la-job-postings?scriptVersionId=16046935#PART-3:-SENTIMENT-ANALYSIS:)

[PART 4: SUGGESTIONS FOR DIVERSITY HIRING IMPROVEMENT](https://www.kaggle.com/vicely07/sentiment-analysis-of-city-of-la-job-postings?scriptVersionId=16046935#PART-4:-SUGGESTIONS-FOR-DIVERSITY-HIRING-IMPROVEMENT:)

[PART 5: VALIDATION OF SUGGESTIONS](https://www.kaggle.com/vicely07/sentiment-analysis-of-city-of-la-job-postings?scriptVersionId=16046935#PART-5:-VALIDATION-OF-SUGGESTIONS:)

PART 6: CONCLUSION

REFERENCES

**Note:**
Since I am working on a jupyter notebook on my personal computer first, there are some incompatible sections when I uploaded my completed project to the Kaggle kernel. Some of the incompatible code sections will be executed by importing existing files from my personal computer. 

## PART 1: JOB POSTING TEXT BULLETIN TO STRUCTURED CSV FILE:

First, we will need to import all the neccessary requirements:

In [ ]:
#These open source libraries are using python 3.6
from __future__ import unicode_literals, print_function
import numpy as np 
import pandas as pd 
import re
import nltk
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
import random
import dateutil.parser as dparser
from datetime import datetime
!pip install datefinder
import datefinder
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import sent_tokenize
from wordcloud import WordCloud, STOPWORDS
from flashtext import KeywordProcessor
import os
import plac
import spacy

Load the data of job bulletins into a List of text, also load the filenames (to be used later), derive the job position from the filename for some preliminary analysis (actual position for data dictionary will be derived from job bulletin)

In [ ]:
def load_jobopening_dataset():

    data_path = "../input/data-science-for-good-city-of-los-angeles/CityofLA/CityofLA/Job Bulletins/"

    texts = []
    positions = []
    file_names=[]
    for fname in sorted(os.listdir(data_path)):
        if fname.endswith('.txt'):
            file_names.append(fname)
            with open(os.path.join(data_path, fname),"rb") as f:
                texts.append(str(f.read()))
                positions.append((re.split(' (?=class)', fname))[0])
    
    #print the length of the List of text, length of file_names and positions and make sure they are all equal
    return (texts,positions,file_names)

#job_data, positions, file_names = load_jobopening_dataset() #This code can't read texts from zip file so I would manually import the raw text files I exported from my personal computer
job_data = pd.read_csv("../input/raw-file/job_data.csv").values.tolist()
positions =  pd.read_csv("../input/raw-file/positions.csv").values.tolist()
file_names =  pd.read_csv("../input/raw-file/file_names.csv").values.tolist()

In [ ]:
str(job_data[0]).replace("\\r\\n"," ").replace("\\\'s","")[:250]

**Provided documents:**

In [ ]:
titles = pd.read_csv("../input/additional-data/job_titles.csv", header=None)
titles.head()

In [ ]:
data_dict=pd.read_csv("../input/additional-data/kaggle_data_dictionary.csv")
data_dict[:5]

**Creating the Parsers:**

In this section, I will create different parsers to extract different information from the text bulletin. Those information are based on the content of sample output file and the field name of data dictionary, which includes:
1. JOB_CLASS_TITLE
2. JOB_CLASS_NO
3. REQUIREMENT_SET_ID
4. REQUIREMENT_SUBSET_ID
5. JOB_DUTIES
6. EDUCATION_YEARS
7. SCHOOL_TYPE
8. EDUCATION_MAJOR
9. EXPERIENCE_LENGTH
10. FULL_TIME_PART_TIME
11. EXP_JOB_CLASS_TITLE
12. EXP_JOB_CLASS_ALT_RESP
13. EXP_JOB_CLASS_FUNCTION
14. COURSE_COUNT
15. COURSE_LENGTH
16. COURSE_SUBJECT
17. MISC_COURSE_DETAILS
18. DRIVERS_LICENSE_REQ
19. DRIV_LIC_TYPE
20. ADDTL_LIC
21. EXAM_TYPE
22. ENTRY_SALARY_GEN
23. ENTRY_SALARY_DWP
24. OPEN_DATE

We would need 24 parsers for 24 columns of the output csv file, the parsers are defined as function below:

In [ ]:
def Position_parser(s):
    title_match=False
    pos = re.findall(r'(.*?)Class Code',s)
    pos1 = re.findall(r'(.*?)Class  Code',s)
    if (len(pos1) > 0):
        pos = pos1
    if (len(pos) > 0):
        job_title= pos[0].replace("b'","").replace("b\"","").replace("'","").replace("\\","").strip()
        for title in titles[0]:
            if (title.replace("'","")==job_title):
                title_match=True
                break
    if(title_match==True):
        return job_title
    else:
        return "Invalid job title" 

In [ ]:
def JobCode_parser(s):
    job_code = 0
    code = re.findall(r'Class Code:(.*?)Open',s)
    if (len(code)>0):
        job_code= int(code[0].strip())
    return job_code

In [ ]:
def OpenDate_parser(s):
    openDateRet=""
    openDate = re.findall(r'Open Date:(.*?)ANNUAL',s)
    openStr=""
    if (len(openDate)>0):
        #print(openDate)
        openDate = openDate[0].strip()
        openStr=re.findall(r'(?:Exam).*',openDate)
        #print(openStr)
    
    matches = list(datefinder.find_dates(openDate))

    if len(matches) > 0:
        for i in range(len(matches)):
            date = matches[i]
            openDateRet=str(date.date())
    return openDateRet,openStr

In [ ]:
def SalaryRange_parser(s):
    salaryRange = re.findall(r'ANNUAL SALARY(.*?)NOTE',s)
    salaryRange_1 = re.findall(r'ANNUAL SALARY(.*?)DUTIES',s)
    salaryRange_2 = re.findall(r'ANNUAL SALARY(.*?)\(flat',s)
    len1=0
    len2=0
    len3=0
    if (len(salaryRange) > 0):
        len1 = len(salaryRange[0])
    if (len(salaryRange_1) > 0):
        len2 = len(salaryRange_1[0])
    if (len(salaryRange_2) > 0):
        len3 = len(salaryRange_2[0])
    if ((len1 > 0) & (len2 > 0)):
        if (len1 < len2):
            salaryRange = salaryRange
        else:
            salaryRange = salaryRange_1
        
    if (len(salaryRange)>0):
        salaryRange = salaryRange[0].strip()
    return salaryRange

In [ ]:
def Qualification_parser(s):
    qual = re.findall(r'REQUIREMENTS/MINIMUM QUALIFICATIONS(.*?)WHERE TO APPLY',s)
    if (len(qual)==0):
        qual = re.findall(r'REQUIREMENT/MINIMUM QUALIFICATION(.*?)WHERE TO APPLY',s)
    if (len(qual)==0):
        qual = re.findall(r'REQUIREMENTS(.*?)WHERE TO APPLY',s)
    if (len(qual)==0):
        qual = re.findall(r'REQUIREMENT(.*?)WHERE TO APPLY',s)
    if (len(qual)>0):
        qual = qual[0].replace("\\'s","'s").strip()
    else:
        qual=""
    return qual

In [ ]:
def Education_parser(s):
    educationMajor=""
    sentences = sent_tokenize(s)
    selected_sentences=[sent for sent in sentences if "major" in word_tokenize(sent)]
    for i in range(len(selected_sentences)):
        major = re.findall(r'major in(.*?),',selected_sentences[i])
        if (len(major)>0):
            educationMajor=major[0].strip()
    return educationMajor

In [ ]:
def EduSemDur_parser(s):
    educationDur=""
    sentences = sent_tokenize(s)
    selected_sentences=[sent for sent in sentences if "semester" in word_tokenize(sent)]
    for i in range(len(selected_sentences)):
        dur = re.findall(r'(.*?)semester',selected_sentences[i])
        #print(dur)
        if (len(dur)>0):
            educationDur=dur[0]+'sememster'
    return educationDur

In [ ]:
def Duties_parser(s):
    duties = re.findall(r'DUTIES(.*?)REQUIREMENT',s)
    jobDuties=""
    if (len(duties)>0):
        jobDuties= duties[0].strip()
    return jobDuties

In [ ]:
def eduYears_parser(s):
    keyword_processor = KeywordProcessor()
    education_yrs=0.0
    keyword_processor.add_keyword('four-year')
    keyword_processor.add_keyword('four years')
    sentences = sent_tokenize(s)
    selected_sentences=[sent for sent in sentences if "degree" in word_tokenize(sent)]
    selected_sentences1=[sent for sent in sentences if "Graduation" in word_tokenize(sent)]

    for i in range(len(selected_sentences)):
        keywords_found = keyword_processor.extract_keywords(selected_sentences[i])
        if (len(keywords_found) > 0):
            education_yrs=4.0
    for i in range(len(selected_sentences1)):
        keywords_found = keyword_processor.extract_keywords(selected_sentences1[i])
        if (len(keywords_found) > 0):
            education_yrs=4.0
   
    return education_yrs

In [ ]:
def expYears_parser(s):
    keyword_processor = KeywordProcessor()
    exp_yrs=0.0
    keyword_processor.add_keyword('four-year')
    keyword_processor.add_keyword('four years')
    keyword_processor.add_keyword('three years')
    keyword_processor.add_keyword('one year')
    keyword_processor.add_keyword('two years')
    keyword_processor.add_keyword('six years')
    sentences = sent_tokenize(s)
    selected_sentences=[sent for sent in sentences if "experience" in word_tokenize(sent)]

    for i in range(len(selected_sentences)):
        keywords_found = keyword_processor.extract_keywords(selected_sentences[i])
        for i in range(len(keywords_found)):
            if keywords_found[i]=='two years':
                exp_yrs=2.0
            elif keywords_found[i]=='one year':
                exp_yrs=1.0
            elif keywords_found[i]=='three years':
                exp_yrs=3.0
            elif keywords_found[i]=='six years':
                exp_yrs=6.0
            elif keywords_found[i]=='four years':
                exp_yrs=4.0
            elif keywords_found[i]=='four-year':
                exp_yrs=4.0
                
    return exp_yrs

In [ ]:
def fullTimePartTime_parser(s):
    keyword_processor = KeywordProcessor()
    fullTimePartTime=""
    keyword_processor.add_keyword('full-time')
    keyword_processor.add_keyword('part-time')
    sentences = sent_tokenize(s)
    selected_sentences=[sent for sent in sentences if "experience" in word_tokenize(sent)]

    for i in range(len(selected_sentences)):
        keywords_found = keyword_processor.extract_keywords(selected_sentences[i])
        for i in range(len(keywords_found)):
            if keywords_found[i]=='full-time':
                fullTimePartTime="FULL TIME"
            elif keywords_found[i]=='part-time':
                fullTimePartTime="PART TIME"
           
                
    return fullTimePartTime

In [ ]:
def DL_parser(s):
    dl = False
    dl_valid = False
    dl_State = ""
    arr = ['driver', 'license']
    keyword_processor = KeywordProcessor()
    keyword_processor.add_keyword('california')
    if any(re.findall('|'.join(arr), qual)):
        dl = True
    if (dl==True):
        sentences = sent_tokenize(s)
        selected_sentence=[sent for sent in sentences if "driver" in word_tokenize(sent)]
        if (len(selected_sentence)>0):
            words = selected_sentence[0].split()
            selected_word = [word for word in words if "valid" in words]
            if len(selected_word)>0:
                dl_valid=True
        for i in range(len(selected_sentence)):   
            keywords_found = keyword_processor.extract_keywords(selected_sentence[i])
            for i in range(len(keywords_found)):
                if keywords_found[i]=='california':
                    dl_State="CA"
                
    if (dl_valid)==True:
        dl_valid="R"
    else:
        dl_valid="P"
    return dl_valid,dl_State

In [ ]:
def Relations_parser(TEXTS, nlp, ENTITY_TYPE):
    entities=[]
    for text in TEXTS:
        doc = nlp(text)
        relations = extract_entity_relations(doc,ENTITY_TYPE)
        for r1, r2 in relations:
            relation=r1.text+"-"+r2.text
            entities.append(relation)
    imp_entities='::::'.join(entities)   
    return imp_entities

In [ ]:
def College_parser(s):
    college=""
    keyword_processor = KeywordProcessor()
    keyword_processor.add_keyword('college or university')
    keyword_processor.add_keyword('college')
    keyword_processor.add_keyword('university')
    keyword_processor.add_keyword('high school')
    sentences = sent_tokenize(s)
    for j in range(len(sentences)):
        sentence = sentences[j]
        keywords_found = keyword_processor.extract_keywords(sentence)
        if (len(keywords_found) > 0):
            for i in range(len(keywords_found) ):
                if (keywords_found[i]=='college or university'):
                    college='college or university'
                    break
                elif (keywords_found[i]=='college'):
                    college='college'
                    break
                elif (keywords_found[i]=='university'):
                    college='university'
                    break
                elif (keywords_found[i]=='high school'):
                    college='high school'
                    break
    

    return college

We also need to create a function to filter a sequence of spans so they don't contain overlaps and another one to merge entities and noun chunks into one token:

In [ ]:
def filter_spans(spans):
    get_sort_key = lambda span: (span.end - span.start, span.start)
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens = set()
    for span in sorted_spans:
        if span.start not in seen_tokens and span.end - 1 not in seen_tokens:
            result.append(span)
            seen_tokens.update(range(span.start, span.end))
    return result


def extract_entity_relations(doc,entity):
    # Merge entities and noun chunks into one token
    seen_tokens = set()
    spans = list(doc.ents) + list(doc.noun_chunks)
    spans = filter_spans(spans)
    with doc.retokenize() as retokenizer:
        for span in spans:
            retokenizer.merge(span)

    relations = []
    for money in filter(lambda w: w.ent_type_ == entity, doc):
        if money.dep_ in ("attr", "dobj"):
            subject = [w for w in money.head.lefts if w.dep_ == "nsubj"]
            if subject:
                subject = subject[0]
                relations.append((subject, money))
        elif money.dep_ == "pobj" and money.head.dep_ == "prep":
            relations.append((money.head.head, money))
    return relations

After defining all the parser funcions, we would need to put the parsers in a loop so it can go through each job bullentin text file (683 postings in total) and extract the information in each posting to place them nicely in a pandas dataframe. The process is as the follows:

In [ ]:
nltk.download('punkt')
nlp = spacy.load("en_core_web_sm")

job_data_export=pd.DataFrame(columns=["FILE_NAME","JOB_CLASS_TITLE","JOB_CLASS_NO","REQUIREMENT_SET_ID",
                                      "REQUIREMENT_SUBSET_ID","JOB_DUTIES",
                                      "EDUCATION_YEARS","SCHOOL_TYPE","EDUCATION_MAJOR","EXPERIENCE_LENGTH","IMP_ENTITIES_QUAL",
                                     "FULL_TIME_PART_TIME","EXP_JOB_CLASS_TITLE","EXP_JOB_CLASS_ALT_RESP"
                                     ,"EXP_JOB_CLASS_FUNCTION","COURSE_COUNT","COURSE_LENGTH","COURSE_SUBJECT"
                                     ,"MISC_COURSE_DETAILS","DRIVERS_LICENSE_REQ","DRIV_LIC_TYPE",
                                     "ADDTL_LIC","EXAM_TYPE","ENTRY_SALARY_GEN","ENTRY_SALARY_DWP","OPEN_DATE","LEGAL_TERMS"])

for i in range(0, len(job_data)-1):

    s = str(job_data[i]).replace("\\r\\n"," ").replace("\\t","")
    position = Position_parser(s)
    qual = Qualification_parser(s)
    DL_valid,DL_state = DL_parser(qual)
    education_yrs = eduYears_parser(qual)
    education_major = Education_parser(qual)
    try:
        job_code = JobCode_parser(s)
        openDate, openStr = OpenDate_parser(s)
    except:
        job_code = "NaN"
        openDate = "NaN"
        openStr = "NaN"
    salaryRange = SalaryRange_parser(s)
    expYrs = expYears_parser(s)
    duties = Duties_parser(s)
    course_length = EduSemDur_parser(qual)
    fullTimePartTime = fullTimePartTime_parser(qual)
    imp_qual_entities=Relations_parser([qual],nlp,"ORG")
    imp_qual_cardinals=Relations_parser([qual],nlp,"CARDINAL")
    imp_legal_terms=Relations_parser([s],nlp,"LAW")
    college = College_parser(qual)
    job_data_export.loc[i,"JOB_CLASS_TITLE"]=position
    job_data_export.loc[i,"FILE_NAME"]=file_names[i]
    job_data_export.loc[i,"DRIVERS_LICENSE_REQ"]=DL_valid
    job_data_export.loc[i,"EDUCATION_YEARS"]=education_yrs
    job_data_export.loc[i,"JOB_CLASS_NO"]=job_code
    job_data_export.loc[i,"OPEN_DATE"]=openDate
    job_data_export.loc[i,"ENTRY_SALARY_GEN"]=salaryRange
    job_data_export.loc[i,"JOB_DUTIES"]=duties
    job_data_export.loc[i,"EXPERIENCE_LENGTH"]=expYrs
    job_data_export.loc[i,"DRIV_LIC_TYPE"]=DL_state
    job_data_export.loc[i,"EDUCATION_MAJOR"]=education_major
    job_data_export.loc[i,"IMP_ENTITIES_QUAL"]=imp_qual_entities
    job_data_export.loc[i,"COURSE_LENGTH"]=course_length
    job_data_export.loc[i,"FULL_TIME_PART_TIME"]=fullTimePartTime
    job_data_export.loc[i,"SCHOOL_TYPE"]=college
    job_data_export.loc[i,"MISC_COURSE_DETAILS"]=imp_qual_cardinals
    job_data_export.loc[i,"LEGAL_TERMS"]=imp_legal_terms
    job_data_export.loc[i,"EXAM_TYPE"]=openStr

It's quite a time-consuming process, let's see our final dataframe:

In [ ]:
job_data_export.head()

1. This is very similar to the sample job class export file as it is used as a template for this text mining process.  After we have the expected csv file, we would need to export it to an actual csv file using the to_csv functions from pandas package.

In [ ]:
job_data_export.to_csv("LA_job_class_export.csv",index=False)

In [ ]:
job_data_export = pd.read_csv("../input/csv-file/LA_job_class_export.csv")
job_data_export.to_csv("LA_job_class_export.csv",index=False)

## PART 2: RAW TEXT ANALYSIS 

In this section, we will mainly perform out text analysis using the NLTK library, which is a top statistical natural language processing toolkit for English language written in the Python.

In [ ]:
nltk.download('stopwords')

**1. How long (in words) is each job posting?**

In [ ]:
word_count = []
for s in job_data:
    word_count.append(len(str(s).split()))

In [ ]:
fig, ax = plt.subplots()
data = np.random.rand(1000)

N, bins, patches = ax.hist(word_count, edgecolor='white', bins=50, linewidth=0)

q1 = 0.25*len(patches)

q3 = 0.75*len(patches)
for i in range(0,int(q1)):
    patches[i].set_facecolor('b')
for i in range(int(q1), int(q3)):
    patches[i].set_facecolor('g')
for i in range(int(q3), len(patches)):
    patches[i].set_facecolor('r')
plt.xlabel('Number of words')
plt.ylabel('Number of samples')
plt.title('Sample length distribution')
plt.show()

In [ ]:
stats = pd.Series(word_count)
q = pd.DataFrame(stats.describe()[3:]).transpose()
q

As we can see in the histogram and the five-number summary table, the job posting is quite lenthy in words. The shortest text has 723 words and the longest reach to 3070 words. Looking at the quantiles, we can see that less than 25% is under 1000 words and a majority of the job postings go over 1273.5. That lead to our next analysis quesion.

**2. Does the length of text effects the readability?**

In our test, we will calculate our readibility score based on the Flesch reading-ease test. The formula for the Flesch reading-ease score (FRES) test is

![image.png](https://storage.googleapis.com/kagglesdsdata/datasets/239843/508454/readscore.png?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1561828906&Signature=Vr%2BYPRetCVGLOUhdp6IM6fSIAHUkLeZQ%2FGlYICj4smiIRM7DVZlbX4aMwpy%2BhDwLiimCScn0j4AcPfjSnW5p4aVBJUh016eBQg5DeqVdIC92WhERD7qsFj1e%2BPMdscBGPIL%2FkVT8mYGse%2BtioiH%2FhoZlNzBte36IuH8NjnFriHHsxR7yf950hDEZ0mgYeF1ldNL1Z5ublPtMDVQqaOvFgxHL%2B1xuLq2qLuV23jklc%2BSRdbO8RNnAHw9LHgciTXTbZojlJSewuHtajPLY%2F9BNUG2xU0sCXQYGCl9%2FCxckSkUByMyoGS4qXqGMNpL1Oxhiy9%2ByaqPA%2F4c%2FLEX8bNWXMQ%3D%3D)

Higher scores indicate material that is easier to read; lower numbers mark passages that are more difficult to read. The following table can be helpful to assess the ease of readability in a document.
The table is an example of values. 

*Note: While the maximum score is 121.22, there is no limit on how low the score can be. A negative score is valid.*

Luckily, the function of calculating Flesch reading-ease score is already built in the textstat package. We are going to use this library to analyze the readability of our raw text:

In [ ]:
!pip install textstat
import textstat
score_list = []
for text in job_data:
    score_list.append(textstat.flesch_reading_ease(str(text)))

In [ ]:
readability=pd.DataFrame(job_data_export["FILE_NAME"])
readability.insert(1, "SCORE", score_list[:len(score_list)-1], True) 
readability.head(10)

In [ ]:
rstats = readability["SCORE"].describe()[3:]
pd.DataFrame(rstats).transpose()

Our max score is 14.43, which fall in the range 0-29 (very difficult to read), and the majority of the text data are negative numbers. This can be intepreted as the job postings are beyond the highest difficulty level to read in Flesch reading-ease test. Beside the lengthy words, the job posters may include many technical jargons and difficult requirements to understand in one reading. This may scare the applicants away since it make the job position seems harder than they actually are. 



**3. How can we fix this readability problem?**

One research study discovered that **the average jobseeker spends just 49.7 seconds reviewing a listing before deciding it’s not a fit.** Therefore, here're some suggestions to fix this problem: 

**Trimming down the job requirements to include only the must-haves:**

Studies show that while men are likely to apply to jobs for which they meet only 60% of the qualifications, women are much more likely to hesitate unless they meet 100% of the listed requirements. When posting a postion, hiring managers might have a long list of qualifications for the role. However, in order to highlight your commitment to inclusion, it’s important to narrrow down to lists to only the must-haves, because many of the qualification are in the "preferred" categories for a perfectly fit candidates (which are rare). By trimming down the qualification, you’ll likely see an increase in applications from female candidates. If some preffered qualifications should be included in the requirement list, they should start with some soften words such as "familiar with", "plus points for", or "if you have one of these skills".

**Avoid using jargon:**

Jargon and unnecessary coporate speaking language should be avoided at all cost because the job postings are written for a larger pool of general audiences. Many jargons includes terms like KPIs, procurement,P&L... Since the candidates can't understand fully the jargon and coporate terms, these word choices can make some candidates feel less qualified for a position. Jargon and corporate language in job postings will also turn off many talented young people since the posting coporation sounds like a conservative, old-fashioned institution. This problem can be easily fixed by rephrasing the words. 

*For example, instead of using technical, broad business terms in job description like "core competency", "detail-oriented", "interpersonal", aim for more universal wording, like “pays attention to details” or “personable with customers.”*

## PART 3: SENTIMENT ANALYSIS:

**1. Cleaning the data:**

In order to perform sentiment analysis, we need to clean up the raw data to extract only the essential features. With that being said, we first need to tokenize the text. Tokenization is the process of breaking down a text paragraph into smaller chunks such as words or sentence. We also need to remove unnecessary words by including them in stopword list. In this case, "city, los angeles, and may" are removed since they shows up as most frequent words and does not carry much weights in our classification process later.

In [ ]:
exclude = set(string.punctuation) 
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')
#
newStopWords = ['city','los','angele','angeles','may']
stop_words.extend(newStopWords)
table = str.maketrans('', '', string.punctuation)

lemma = WordNetLemmatizer()
porter = PorterStemmer()

We also perform Lexicon normalization. The normalization process considers another type of noise in the text. For example, "connection", "connected", "connecting" are reduced to the common word "connect". It reduces derivationally related forms of a word to a common root word.

In [ ]:
def normalize_document(doc):
    #replace newline and tab chars
    doc = doc.replace("\\r\\n"," ").replace("\\\'s","").replace("\t"," ") #.split("b'")[1]
    # tokenize document
    tokens = doc.split()
    # remove punctuation from each word
    tokens = [w.translate(table) for w in tokens]
    # convert to lower case
    lower_tokens = [w.lower() for w in tokens]
    #remove spaces
    stripped = [w.strip() for w in lower_tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter stopwords out of document
    filtered_tokens = [token for token in words if token not in stop_words]
    #normalized = " ".join(lemma.lemmatize(word) for word in filtered_tokens)
    #join the tokens back to get the original doc
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)
#apply the text normalization to list of job positions
norm_positions=[]
for text_sample in positions:
    norm_positions.append(normalize_document(str(text_sample)))
#apply the text normalization to list of job ads
norm_corpus=[]
for text_sample in job_data:
    norm_corpus.append(normalize_document(str(text_sample)))

**2. Charts of most frequent words:**

After we cleaning up the text to extract only the most important information, we are going to plot our normalized texts for further analysis. First, we would plot a word clouds. It is a good visualization to show the most frequently used words in the job posting text. The more frequent a specific word appears in a source of textual data, the bigger and bolder it appears in the word cloud. We look at the frequent words because they appear on every single job posting. Hence, our analysis will have the bigger influence on each individual job posting. 

In [ ]:
full_norm_corpus=' '.join(norm_corpus)
stopwords = set(STOPWORDS)
stopwords.update(["class", "code"])

wordcloud = WordCloud(background_color='white', stopwords=stopwords,max_words=100).generate(full_norm_corpus)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

From the word cloud, we can see some positive and negative keywords showing up:

**Positive words:** "disability accommodation", "civil service", "open competitive", "equal employment", "employment opportunity".

Those terms provoke a positive feeling that the employers really value diversity and welcome applicants from all backgrounds, genders and situations to apply. 

**Negative words:** "written test", "must received", "eligible list", "minimum qualification", "service rule", "reserve list"

These terms, in contrast, give a negative impression that the employers follow very strict rules and have many requirements for the job opening. That will discourage many applicants. 

We will create a n-grams function to extract longer terms and plot top 30 most frequent terms:

In [ ]:
def ngrams(sample_texts, ngram_range, num_ngrams=30):
    """Plots the frequency distribution of n-grams.

    # Arguments
        samples_texts: list, sample texts.
        ngram_range: tuple (min, mplt), The range of n-gram values to consider.
            Min and mplt are the lower and upper bound values for the range.
        num_ngrams: int, number of n-grams to plot.
            Top `num_ngrams` frequent n-grams will be plotted.
    """
    # Create args required for vectorizing.
    kwargs = {
            'ngram_range': ngram_range,
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': 'word',  # Split text into word tokens.
    }
    vectorizer = CountVectorizer(**kwargs)
    vectorized_texts = vectorizer.fit_transform(sample_texts)

    # This is the list of all n-grams in the index order from the vocabulary.
    all_ngrams = list(vectorizer.get_feature_names())
    num_ngrams = min(num_ngrams, len(all_ngrams))
    ngrams = all_ngrams[:num_ngrams]

    # Add up the counts per n-gram ie. column-wise
    all_counts = vectorized_texts.sum(axis=0).tolist()[0]

    # Sort n-grams and counts by frequency and get top `num_ngrams` ngrams.
    all_counts, all_ngrams = zip(*[(c, n) for c, n in sorted(
        zip(all_counts, all_ngrams), reverse=True)])
    ngrams = list(all_ngrams)[:num_ngrams]
    counts = list(all_counts)[:num_ngrams]
    return ngrams, counts


ngrams4, counts4 = ngrams(norm_corpus,ngram_range=(4, 4))

In [ ]:
# Fixing random state for reproducibility
idx = np.arange(30)
np.random.seed(19680801)
plt.rcdefaults()
fig, ax = plt.subplots()
#horizontal
ax.barh(idx, counts4, align='center', color='g')
ax.set_yticks(idx)
ax.set_yticklabels(ngrams4, rotation=0, fontsize=8)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Frequencies',fontsize="12")
ax.set_title('Frequency distribution of n-grams',fontsize="12")

plt.show()

We will use the textblob library, which has a pre-trained machine learning model, to perform our sentiment analysis. An example of the classification can be:

In [ ]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import nltk
#nltk.download('movie_reviews')
#nltk.download('punkt')

text          = "I only hire male applicants" 

sent          = TextBlob(text)
# The polarity score is a float within the range [-1.0, 1.0]
# where negative value indicates negative text and positive
# value indicates that the given text is positive.
polarity      = sent.sentiment.polarity
# The subjectivity is a float within the range [0.0, 1.0] where
# 0.0 is very objective and 1.0 is very subjective.
subjectivity  = sent.sentiment.subjectivity

sent          = TextBlob(text, analyzer = NaiveBayesAnalyzer())
classification= sent.sentiment.classification
positive      = sent.sentiment.p_pos
negative      = sent.sentiment.p_neg


dict1 = {'Polarity': polarity,'Subjectivity': subjectivity, 'Classification': classification, 'Posititve': positive, 'Negative': negative}
df1 = pd.Series(dict1)
df1


**Polarity:** simply means emotions expressed in a sentence. The emotional scale is: Negative < 0, Neutral = 0, Positive > 0.

**Subjectivity:** sentence expresses some personal feelings, views, or beliefs. We will not use this as a metric in our analysis.

**Classification:** By default, the NaiveBayesClassifier uses a simple feature extractor that indicates which words in the training set (movie reviews datset) are contained in a document. The textblob model take into consideration the tone and content of the text and can classify with an accuracy of 87%. 

We now will make the two functions for classification and polarity from Textblob library for later analysis on n-grams:

In [ ]:
def pos_neg_classify(text):
    sent          = TextBlob(text)
    # The polarity score is a float within the range [-1.0, 1.0]
    # where negative value indicates negative text and positive
    # value indicates that the given text is positive.
    polarity      = sent.sentiment.polarity
        
    # The subjectivity is a float within the range [0.0, 1.0] where
    # 0.0 is very objective and 1.0 is very subjective.
    subjectivity  = sent.sentiment.subjectivity
    
    sent          = TextBlob(text, analyzer = NaiveBayesAnalyzer())
    classification= sent.sentiment.classification
    pos_score = round(sent.sentiment.p_pos,2)
    neg_score = round(sent.sentiment.p_neg,2)
    if pos_score > neg_score:
        clas = 'POSITIVE'
    elif pos_score < neg_score:
        clas = 'NEGATIVE'
    else:
        clas = 'NEUTRAL'
    return clas

def polarity_classify(text):
    sent          = TextBlob(text)
    # The polarity score is a float within the range [-1.0, 1.0]
    # where negative value indicates negative text and positive
    # value indicates that the given text is positive.
    polarity      = round(sent.sentiment.polarity, 1)
    
    if polarity > 0:
        pol = 'EMOTIONAL POSITIVE'
    elif polarity < 0:
        pol = 'EMOTIONAL NEGATIVE'
    else:
        pol = 'EMOTIONAL NEUTRAL'
        
    return pol

Let's use the functions to classify whether our 4-gram term are positive ore negative:

In [ ]:
pd.set_option('display.max_colwidth', -1)
df_4 = pd.DataFrame(ngrams4, columns=['N-grams Sentence'])
class_list4 = []
pol_list4 = []
for text in ngrams4:
    classification = pos_neg_classify(text)
    polarity = polarity_classify(text)
    class_list4.append(classification)
    pol_list4.append(polarity)
df_4["Content"] = class_list4
df_4["Polarity"] = pol_list4
df_4.to_csv("ngrams4sentiment.csv",index=False)
df_4

Let's see which 4-grams words are classified negative according to our machine learning model:

In [ ]:
df_4_neg = df_4[df_4["Content"]=="NEGATIVE"]
df_4_neg.to_csv("negative.csv",index=False)
df_4_neg

As we can see, the terms of "minimum qualifcations", "acquired" and "origin sex age" are considered to be negative in content and all of them are neutral in emotion. Let's furture analyze those negative 4-gram terms by finding these terms in a list of 10-grams. This way, we can the full context where the terms are in and can have a better analysis of thoese terms:

In [ ]:
neg_4grams = df_4_neg["N-grams Sentence"]
neg_list = []
class_list10 = []
pol_list10 = []
ngrams10, counts10 = ngrams(norm_corpus,ngram_range=(10, 10))   
for text in ngrams10:
    for neg_word in neg_4grams:
        neg_index = text.find(neg_word)    
        if neg_index != -1:
            if text not in neg_list:
                neg_list.append(text)
                classification = pos_neg_classify(text)
                polarity = polarity_classify(text)
                class_list10.append(classification)
                pol_list10.append(polarity)
                
df_neg = pd.DataFrame(neg_list, columns=['N-grams Sentence'])
df_neg["Content"] = class_list10
df_neg["Polarity"] = pol_list10
df_neg.to_csv("Ngrams30sentiment.csv",index=False)
df_neg

When putting the negative 4-gram terms in context of 10-gram terms, they turn out to be all positive in content. However, we can see that the phrase "subject review ensure minimum qualifications met candidates disqualified time determined" is considered to be negative in emotion. It is proved to be true since the phrase sounds very demanding and make a large number of job seekers feel less confident to apply for the position. This phrase should definitely omitted from the job posting. 

## PART 4: SUGGESTIONS FOR DIVERSITY HIRING IMPROVEMENT:

**According to our findings, the job postings include many negative words in the requirement section. Therefore, below are 5 additional suggestions that can fix the sentiment problems in order to improve the diversity hiring at city of LA:**

**1. Re-word your job posting to attract diversity:**

The current job postings include a big part talking about requirements, standard tests and applying process. It's like a teacher gives a long list of homeworks and assignments on the first day of class. Hence, those information will scare many job seekers away. As we mentioned in part 2, we should cut down the length of text to increase the readibility of the job postings. The testing requirements and application processes can definitely cut down from our posting. These processes can be included in a follow-up email to candidates who actually apply to the job.

In addition, many studies has found that the language you use in a job description can significantly attract or turn off diverse candidates from applying to the listed position. To attract more female candidates, avoid using too many “masculine” words (ambitious, dominate, challenging ...) in the job postings. 

**2. Show your existing workplace diversity (or the diversity you aspire to):**

Glassdoor found that 67% of job seekers use diversity is an important factor when considering companies and job offers. One of the biggest effective way to increase workplace diversity is showing diversity at your workplace itself. 
In February of 2018, Mayor Eric Garcetti today did a really good job in creating a social media campaign that highlighted diversity in the Los Angeles Police Department and encourage more Angelenos to consider careers in the LAPD. One of the photo banner from this campaign shows different LAPD employees holding paper signs telling their unique personal stories (the picture attached below). The city of LA can work on a similar campaign with real employees from each department. 

![](https://storage.googleapis.com/kagglesdsdata/datasets/239299/507558/Diversity.png?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1561828777&Signature=m4cHD6OuJzqKEHOrpi7J6UiWsGLaNNtN26WCE3OSzmlH9bWOGMqQOkpmLT%2BvlSOQx5Arh%2FOMEWXqXTqY8kwnWiUoMORXkInoEoo9GoJ%2Bs0M35cqLJVgyS5O8rWeLTaGT072H1jMsrydDWteQ4R%2FDdPTdfnn6gbM74oiSDkGCjUovmmqqDSeDsnDaKV2gd9pQosnxx0NKdE5yyC%2B9CnBUV6GNwi%2BScBIg4bfpIONMaNOS7Hrxima9BpmhoAHJ6%2FCeSZWVCSY8PjFF3HnD7alEzg2RNfdr8y0yDgEvgcMJYx2GqpRHjgUwvTpgllQJWwS4J4P7h8XluNboiFnO%2Bl31hA%3D%3D)

**3. Encourage referrals from minority employees:**

Current employee at your organization usually has professional networks are made up of people from the similar demographic. You can leverage this diverse network advantage by encouraging minority employees (with or without bonuses) to give referrals. A great amount of referrals from minority employee will help increase your diversity hiring drastically.


**4. Call out inclusive benefits like parental leave and childcare subsidies:**

Beside salary, company benifits is the second key point that an average job seekers scan through when they read a job posting. Thereforem bring those two important pieces of information to the top of the page. Good salary, paid parental leave, childcare subsidies, paid family sick time, and even health insurance can go a long way toward supporting and attracting diversity and inclusion. It also shows the company actually care about the well-being of their employees. 

**5. Offer workplace flexibility:**

Research has found a strong correlation between employees’ quitting and a long commute distance. Therefore, offering flexible work hours and even work from home option does not only attract more diverse candidates, it also helps reduce some operating expenses. 

## PART 5: VALIDATION OF SUGGESTIONS:

It's easier to say than done. Let's take all our suggestions into consideration and create a revised job postings. Below is a revised job sample:

In [ ]:
old_jobposting = open('../input/sample-text/ADMINISTRATIVE ANALYST 1590 060118.txt', 'r')
content1 = old_jobposting.read()

In [ ]:
!pip install textstat

In [ ]:
import textstat
print("Statistics of Revised Job Posting:")
numword = textstat.lexicon_count(content1, removepunct=True)
score = textstat.flesch_reading_ease(content1)
class1 = pos_neg_classify(content1)
pol1         = polarity_classify(content1)
contentstat1 = pd.Series({"Word counts": numword, "Readbility Score": score, "Content": class1, "Polarity": pol1})
contentstat1

In [ ]:
revised_jobposting = open('../input/revised-text-posting/Revised posting.txt', 'rb')
content2 = str(revised_jobposting.read()).replace("\\r\\n"," ").replace("\\\'s","")
print(content2)

In [ ]:
print("Statistics of Revised Job Posting:")
numword = textstat.lexicon_count(content2, removepunct=True)
score = textstat.flesch_reading_ease(content2)
class2 = pos_neg_classify(content2)
pol2         = polarity_classify(content2)
contentstat2 = pd.Series({"Word counts": numword, "Readbility Score": score, "Content": class2, "Polarity": pol2})
contentstat2

As we can see, the a negative readability score (-8.29) from original postings increases to a college-level readability score of 4.68 in the revised posting. The contents in both remain positive with the emotional tone is changed from neutral to positive in the revised version. Therefore, our suggestion does help improve the sample job posting.

**Since our suggestion works well in improving the readbility and semtiment of the original text, let's put all our above sugestions from part 3 into a nicer and cleaner posting. Below is my detailed comments on what I changed from the original text:**

![](![image.png](attachment:image.png)https://storage.googleapis.com/kagglesdsdata/datasets/239299/507558/RevisedPostingwithComment.png?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1561320217&Signature=Ege8Bw%2FgzPqqGGgdJQolRpzRvEl%2FQmvkwDWoybKJ05n15y4X5sQqKjNgZDoDePzFYeeM6wRwzR1KSDqKC6tBg40FJ58KTEjuBVi16CdVkds23a3lfbsEU27%2FqkJZoz4pHSL%2Fi7CSJyyd%2FQyWPu28Um%2FrbIKupArrFXJqZnm2%2FddjKZUQhuvTl3lsm6Y3xRUVAnUn10yCOzTC8p%2FaAOOw5fc3tHrn8Ugzgf5deaM4aEkTo71M11nVjBnfpUPh7m%2BBxMH9J8tkLmjiDBvAMWF%2Fbyfq410QT6bW5E9lpf16ducG9xH%2Bc%2BsOFonxrY3%2FfIoD8eYA%2F7Ln6WahB%2Bsjnx0EiA%3D%3D)

**Original job positing:**

![Original posting](https://storage.googleapis.com/kagglesdsdata/datasets/240430/509531/original%20posting.png?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1561828837&Signature=K3IyMbL9qSttVVoaJce7zJ6k%2FB0hORZleMiT5yoE1ab1AgfQAMW0YUxKhnWsGa%2FYKk0GsCblJzHxa7s8qGMtjGcaZSTiGqANqcrq6NvEx%2B7SvFfiias7EtPec5hvBP3%2BXxA2P2wg2JJKVHC3H9ZwOtodm1IS1R4nD4KbWWP9%2BDF9hNhzGI0f4nSrwUITpZTfUUL8Pm8QmYrcOuqyYV%2BxQtK%2FTafKsGBvr6V7rAmOmOE3n6oQHPb8gB0BSdfH7%2BZJYyg2qE%2B85rXlr3%2FWOALqO6%2FO86VJz7FVpkUYRkKOH6n%2FOzSXtp7QcEFxtOm7rxf%2FAoGtg952jG39bKFXMYuQGw%3D%3D)

**Our final revised job posting:**

![Revised posting](https://storage.googleapis.com/kagglesdsdata/datasets/239299/507558/RevisedPosing.png?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1561828811&Signature=FXMhjTk9TVKvKsRkH%2BP32rSoza8H8zkKDA%2BARzHJZKvCx1vYQa29AsKaCX24QhOw2EAwr9Xusb%2FrODchU7lK%2FMQVW0KUwNvHT9FN%2B10tnb4OW1O5ie9W2aVp0y%2Ftos%2BFm95yBm1xHvEheMnyjleQn7gN6tKTX%2B7crrh4ALrbZQdc5qUq6%2BP2SRth2cldEVie8sj%2BsfnOgekhkIWGHYxEtWn6eQvPM8fknHLjwwuzUdHBWsX%2FhOPRcaGOBVGmWpAQF281rwFqN8yB8nB%2B3bx5l%2F%2BrksBb5COvfW8MQ%2B0AySOhyvBx0gCavSFGgz%2FogptwPMjS4D224p55MCWI0%2BqDyA%3D%3D)

Looking at the above job posting and comparing to the text file of the same position from the dataset, which ones would you refer to read and feel more comfortable to take action to apply for? You may already have the answer. 

## PART 6: CONCLUSION:

In summary, we first extract the information from the plain text job postings to create a csv table. Then, we use this new csv table and the normalized text files for the purpose of calculating ngrams of most frequently appeared words, plotting the distribution of text length, word clouds and sentiment analysis of the ngrams. From our analytics, we can see that the job posting texts are lenghty in words, have low readability scores, contain many negative words. 
By focusing on fixing the length of the text, filtering out the negative words, adding a some other suggestions on promoting the company's mission and commitment to diversity hiring, we then validate our suggestions by running the metrics from our analysis on the new text. In our last step, we add graphic and reformat the plain text into a neat and clean job posting. This new job posting can play a key part helping the city of LA to meet their goal in diversity hiring the future. 

## REFERENCES:

1. Flesch–Kincaid readability tests. (2019, May 31). Retrieved from https://en.wikipedia.org/wiki/Flesch–Kincaid_readability_tests
2. Natural Language Toolkit. (n.d.). Retrieved from https://www.nltk.org/
3. Pellarolo, M., & Pellarolo, M. (2018, February 20). Naive Bayes for Sentiment Analysis. Retrieved from https://medium.com/@martinpella/naive-bayes-for-sentiment-analysis-49b37db18bf8 
4. DataScience_CityofLA. (n.d.). Retrieved from https://www.kaggle.com/anirbank/datascience-cityofla
5. Simplified Text Processing. (n.d.). Retrieved from https://textblob.readthedocs.io/en/dev/
6. Garcetti, & Lapd. (2018, February 23). Mayor Garcetti and LAPD launch recruitment campaign focused on diversity. Retrieved from https://www.lamayor.org/mayor-garcetti-and-lapd-launch-recruitment-campaign-focused-diversity